In [18]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Preprocesamiento

In [5]:
df = pd.read_csv('../Data/df.csv', index_col=0)
df.head(3)

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_low,Product_importance_medium,Gender_M
ID,,,,,,,,,,,,,,,,
1,4,2,177,3,44,1233,1,False,False,True,False,False,False,True,False,False
2,4,5,216,2,59,3088,1,False,False,False,True,False,False,True,False,True
3,2,2,183,4,48,3374,1,False,False,False,False,False,False,True,False,True


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10999 entries, 1 to 10999
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   Customer_care_calls        10999 non-null  int64
 1   Customer_rating            10999 non-null  int64
 2   Cost_of_the_Product        10999 non-null  int64
 3   Prior_purchases            10999 non-null  int64
 4   Discount_offered           10999 non-null  int64
 5   Weight_in_gms              10999 non-null  int64
 6   Reached.on.Time_Y.N        10999 non-null  int64
 7   Warehouse_block_B          10999 non-null  bool 
 8   Warehouse_block_C          10999 non-null  bool 
 9   Warehouse_block_D          10999 non-null  bool 
 10  Warehouse_block_F          10999 non-null  bool 
 11  Mode_of_Shipment_Road      10999 non-null  bool 
 12  Mode_of_Shipment_Ship      10999 non-null  bool 
 13  Product_importance_low     10999 non-null  bool 
 14  Product_importance_medium  

In [8]:
df['Reached.on.Time_Y.N'].value_counts()

Reached.on.Time_Y.N
1    6563
0    4436
Name: count, dtype: int64

In [9]:
X = df.drop(columns=['Reached.on.Time_Y.N'])
y = df['Reached.on.Time_Y.N']

In [19]:
from flaml import AutoML
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

def evaluate_sampling(X_train, y_train, X_test, y_test, sampler_name, sampler):
    # Aplicar sampling
    X_res, y_res = sampler.fit_resample(X_train, y_train)
    print(f"\n🔹 {sampler_name}: distribución original {Counter(y_train)} → nueva {Counter(y_res)}")

    # Entrenar modelo
    automl = AutoML()
    automl.fit(X_res, y_res, task="classification", time_budget=60, verbose=0)

    # Predicciones
    y_pred = automl.predict(X_test)

    # Métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='weighted')
    rec = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"✅ Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")
    print(classification_report(y_test, y_pred, zero_division=0))
    print(f"Mejor modelo: {automl.best_estimator}")

# --- Random UnderSampling ---
rus = RandomUnderSampler(random_state=42)
evaluate_sampling(X_train, y_train, X_test, y_test, "RandomUnderSampler", rus)

# --- Random OverSampling ---
ros = RandomOverSampler(random_state=42)
evaluate_sampling(X_train, y_train, X_test, y_test, "RandomOverSampler", ros)



🔹 RandomUnderSampler: distribución original Counter({1: 5235, 0: 3564}) → nueva Counter({0: 3564, 1: 3564})
✅ Accuracy: 0.6873 | Precision: 0.8146 | Recall: 0.6873 | F1: 0.6783
              precision    recall  f1-score   support

           0       0.56      0.99      0.71       872
           1       0.98      0.49      0.65      1328

    accuracy                           0.69      2200
   macro avg       0.77      0.74      0.68      2200
weighted avg       0.81      0.69      0.68      2200

Mejor modelo: extra_tree

🔹 RandomOverSampler: distribución original Counter({1: 5235, 0: 3564}) → nueva Counter({1: 5235, 0: 5235})
✅ Accuracy: 0.6486 | Precision: 0.6625 | Recall: 0.6486 | F1: 0.6521
              precision    recall  f1-score   support

           0       0.55      0.65      0.59       872
           1       0.74      0.65      0.69      1328

    accuracy                           0.65      2200
   macro avg       0.64      0.65      0.64      2200
weighted avg       0.

# LazyClassifier

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 5235, number of negative: 3564
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001127 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 558
[LightGBM] [Info] Number of data points in the train set: 8799, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.594954 -> initscore=0.384483
[LightGBM] [Info] Start training from score 0.384483
                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
SVC                                0.68               0.72     0.72      0.67   
QuadraticDiscriminantAnalysis      0.66               0.72     0.72      0.65   
GaussianNB                         0.65               0.71     0.71      0.63   
BaggingClassifier                  0.68               0.69     0.69      0.68   
NearestCentroid            

In [13]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SVC,0.68,0.72,0.72,0.67,5.40
QuadraticDiscriminantAnalysis,0.66,0.72,0.72,0.65,0.02
GaussianNB,0.65,0.71,0.71,0.63,0.01
BaggingClassifier,0.68,0.69,0.69,0.68,0.37
NearestCentroid,0.65,0.69,0.69,0.65,0.10
BernoulliNB,0.67,0.68,0.68,0.67,0.03
LGBMClassifier,0.67,0.68,0.68,0.67,1.34
AdaBoostClassifier,0.67,0.68,0.68,0.68,0.68
RandomForestClassifier,0.66,0.67,0.67,0.67,1.91


# FLAML

In [ ]:
from flaml import AutoML

automl = AutoML()
automl.fit(X_train, y_train, task="classification", time_budget=60, verbose=0)

[flaml.automl.logger: 10-21 21:21:47] {1752} INFO - task = classification
[flaml.automl.logger: 10-21 21:21:47] {1763} INFO - Evaluation method: cv
[flaml.automl.logger: 10-21 21:21:47] {1862} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 10-21 21:21:47] {1979} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'lrl1']
[flaml.automl.logger: 10-21 21:21:47] {2282} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 10-21 21:21:48] {2417} INFO - Estimated sufficient time budget=2795s. Estimated necessary time budget=65s.
[flaml.automl.logger: 10-21 21:21:48] {2466} INFO -  at 0.6s,	estimator lgbm's best error=0.2564,	best estimator lgbm's best error=0.2564
[flaml.automl.logger: 10-21 21:21:48] {2282} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 10-21 21:21:48] {2466} INFO -  at 0.8s,	estimator lgbm's best error=0.2558,	best estimator lgbm's best error=0.2558
[flaml.automl.logger: 10-21 

In [17]:
from flaml import AutoML
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Entrenamiento
automl = AutoML()
automl.fit(X_train, y_train, task="classification", time_budget=60, verbose=0)

# Predicciones
y_pred = automl.predict(X_test)

# Probabilidades (opcional, si tu modelo las soporta)
y_proba = automl.predict_proba(X_test)

# Métricas principales
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall: {recall:.4f}")
print(f"✅ F1-score: {f1:.4f}")

# Reporte más completo
print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_test, y_pred))

# Mejor modelo encontrado
print("\n--- Mejor modelo ---")
print(automl.best_estimator)
#print(automl.best_model)
print(automl.best_config)
print(f"Best accuracy (cv): {automl.best_loss:.4f}")


✅ Accuracy: 0.6036
✅ Precision: 0.3644
✅ Recall: 0.6036
✅ F1-score: 0.4544

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       872
           1       0.60      1.00      0.75      1328

    accuracy                           0.60      2200
   macro avg       0.30      0.50      0.38      2200
weighted avg       0.36      0.60      0.45      2200


--- Confusion Matrix ---
[[   0  872]
 [   0 1328]]

--- Mejor modelo ---
lgbm
{'n_estimators': 5, 'num_leaves': 5, 'min_child_samples': 35, 'learning_rate': np.float64(0.0599975222148211), 'log_max_bin': 8, 'colsample_bytree': np.float64(0.8164760447739485), 'reg_alpha': 0.0009765625, 'reg_lambda': np.float64(0.25229910134609573)}
Best accuracy (cv): 0.2546
